In [3]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'june'
new_month = 'july'

In [6]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2020.xlsx',\
                         sheetname='assets',index_col='Ticker')

In [7]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [8]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [9]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [10]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [11]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [12]:
a = df.index.tolist()

In [15]:
a.index('zrx')

129

In [16]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index[129:131]:
    try:
        print(t)
        metrics = get_messari_data(t)
        print(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

zrx
zrx
rvn
rvn


In [17]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [18]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [19]:
t = 'BNT'
metrics = get_messari_data(t)
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Est Volume')] = metrics['Est Volume']
except:
    pass
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
   pass

KeyError: 'Available Supply'

In [213]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [214]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [215]:
#read in other data sources
onchainfx_db = df

In [216]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

143
142
143
142


In [217]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOT' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [218]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [219]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [1]:
onchainfx_db.loc['BNT']

NameError: name 'onchainfx_db' is not defined

In [221]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [222]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [223]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [224]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

ECN  no other data!


In [225]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,3.837850e+07,3.837850e+07,1.0,1.844675e+11,0.000208,1.844675e+11,7.262250e+06,1,0.189227,1,0.0,1.0,0.0,2.0,0
CTXC,2.817287e+07,2.817287e+07,1.0,2.539879e+08,0.110922,2.539879e+08,3.291449e+09,1,116.830439,1,0.0,2.0,0.0,2.0,0
DENT,1.955851e+07,1.955851e+07,1.0,1.000000e+11,0.000196,1.000000e+11,8.622068e+07,1,4.408347,1,0.0,3.0,0.0,2.0,0
DRGN,2.930215e+07,2.930215e+07,1.0,3.392024e+08,0.086385,3.392024e+08,1.269443e+07,1,0.433225,1,0.0,3.0,0.0,2.0,0
ETN,5.345356e+07,5.345356e+07,1.0,1.014576e+10,0.005269,1.014576e+10,1.262963e+08,1,2.362730,1,0.0,3.0,0.0,2.0,0
GRIN,4.940638e+08,4.940638e+08,1.0,9.771011e+08,0.505642,9.771011e+08,1.409322e+09,1,2.852510,1,0.0,1.0,0.0,2.0,0
KCS,6.964020e+07,6.964020e+07,1.0,8.125028e+07,0.857107,8.125028e+07,1.047075e+09,1,15.035493,1,0.0,2.0,0.0,2.0,0
MKR,4.555395e+08,4.555395e+08,1.0,9.966470e+05,457.072017,9.966470e+05,1.835570e+09,1,4.029443,1,0.0,3.0,0.0,2.0,0
R,5.105801e+06,5.105801e+06,1.0,4.844500e+08,0.010539,4.844500e+08,2.087286e+08,1,40.880689,1,0.0,3.0,0.0,2.0,0
HT,9.288990e+08,9.288990e+08,1.0,2.210633e+08,4.201959,2.210633e+08,1.477882e+10,1,15.910041,1,0.0,3.0,1.0,2.0,0


In [226]:
#eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
eligible_db = eligible_db.drop('ATOM')

In [227]:
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,3.837850e+07,3.837850e+07,1.0,1.844675e+11,0.000208,1.844675e+11,7.262250e+06,1,0.189227,1,0.0,1.0,0.0,2.0,0
BLOCK,7.208400e+06,7.208400e+06,1.0,6.890570e+06,1.046125,6.890570e+06,NaN,1,NaN,0,1.0,2.0,0.0,2.0,0
CENNZ,6.880347e+07,6.880347e+07,1.0,8.511038e+08,0.080840,8.511038e+08,5.569535e+06,1,0.080948,0,0.0,3.0,0.0,1.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,2.817287e+07,2.817287e+07,1.0,2.539879e+08,0.110922,2.539879e+08,3.291449e+09,1,116.830439,1,0.0,2.0,0.0,2.0,0
DENT,1.955851e+07,1.955851e+07,1.0,1.000000e+11,0.000196,1.000000e+11,8.622068e+07,1,4.408347,1,0.0,3.0,0.0,2.0,0
DRGN,2.930215e+07,2.930215e+07,1.0,3.392024e+08,0.086385,3.392024e+08,1.269443e+07,1,0.433225,1,0.0,3.0,0.0,2.0,0
ECN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,NaN,NaN,0.0,0
EDGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,1.0,1.0,0
ETHOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,1.0,1.0,0


In [228]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

In [229]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,1.922338e+11,1.922338e+11,1.0,2.098634e+07,9159.950257,2.098634e+07,3.296533e+12,1,17.148560,1,1.0,1.0,0.0,3.0,1,1
ETH,3.105530e+10,3.105530e+10,1.0,1.351356e+08,229.808385,1.351356e+08,1.262940e+12,1,40.667468,1,1.0,2.0,0.0,3.0,1,2
XRP,1.830955e+10,1.830955e+10,1.0,1.000000e+11,0.183095,1.000000e+11,2.219367e+11,1,12.121365,1,1.0,2.0,0.0,3.0,1,3
BCH,4.837380e+09,4.837380e+09,1.0,2.098243e+07,230.544305,2.098243e+07,2.296500e+11,1,47.474050,1,1.0,1.0,0.0,3.0,1,4
XTZ,4.795733e+09,4.795733e+09,1.0,1.923801e+09,2.492843,1.923801e+09,2.041531e+10,1,4.256974,1,1.0,2.0,0.0,3.0,1,5
LINK,4.727312e+09,4.727312e+09,1.0,1.000000e+09,4.727312,1.000000e+09,6.969293e+10,1,14.742613,1,1.0,3.0,1.0,3.0,1,6
LTC,3.620491e+09,3.620491e+09,1.0,8.388054e+07,43.162464,8.388054e+07,2.899839e+11,1,80.095192,1,1.0,1.0,0.0,3.0,1,7
BSV,3.515343e+09,3.515343e+09,1.0,2.098214e+07,167.539765,2.098214e+07,1.791010e+11,1,50.948378,1,1.0,1.0,0.0,3.0,1,8
EOS,3.390498e+09,3.390498e+09,1.0,1.366412e+09,2.481315,1.366412e+09,2.581663e+11,1,76.144062,1,1.0,2.0,0.0,3.0,1,9
ADA,3.389724e+09,3.389724e+09,1.0,4.201402e+10,0.080681,4.201402e+10,3.087555e+10,1,9.108573,1,1.0,2.0,0.0,3.0,1,10


In [230]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained EOS
Retained XLM
10 full! Do not add  ADA
10 full! Do not add  TRX


In [231]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained ZIL
Retained XEM
Retained ZRX
Retained DCR
20 full! Do not add  LSK
20 full! Do not add  SC
20 full! Do not add  THETA
20 full! Do not add  WAVES


In [232]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained STORJ
Retained AE
Retained RCN
Retained MANA
Retained GNT
Retained ARDR
Retained ELF
Retained GXC
Retained MAID
Retained WAN
40 full! No room for FUN
40 full! Do not add  BNT
40 full! Do not add  WAX
40 full! Do not add  STRAT
40 full! Do not add  ANT
40 full! Do not add  POLY
40 full! Do not add  EDO
40 full! Do not add  POWR
40 full! Do not add  RDD
40 full! Do not add  CVC


In [233]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [234]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [235]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Total adds ['LRC']
Total deletes ['FUN']
Total count members: 70

Currency adds []
Currency deletes []
Currency count members: 19

20 adds []
20 deletes []
20 count members: 20

Platform adds []
Platform deletes []
Platform count members: 26

ETH adds ['LRC']
ETH deletes ['FUN']
ETH count members: 20

40 adds ['LRC']
40 deletes ['FUN']
40 count members: 40

10 adds []
10 deletes []
10 count members: 10

Application adds ['LRC']
Application deletes ['FUN']
Application count members: 25

Sector count members: 70


In [236]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])